It happens all the time: someone gives you data containing malformed strings, Python,
lists and missing data. How do you tidy it up so you can get on with the analysis?
Take this monstrosity as the DataFrame to use in the following puzzles:

In [13]:
import numpy as np
import pandas as pd

In [54]:
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN', 'londON_StockhOlm',
'Budapest_PaRis', 'Brussels_londOn'],
'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )',
'12. Air France', '"Swiss Air"']})

Some values in the the FlightNumber column are missing. These numbers are meant
to increase by 10 with each row so 10055 and 10075 need to be put in place. Fill in
these missing numbers and make the column an integer column (instead of a float
column).

In [55]:
#Interpolating the missing values linearly and converting it to integer
df['FlightNumber'] = df['FlightNumber'].interpolate(method='linear').astype(int) 

In [56]:
#We can see FlightNumber data type is changed to Int
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
Airline         5 non-null object
FlightNumber    5 non-null int32
From_To         5 non-null object
RecentDelays    5 non-null object
dtypes: int32(1), object(3)
memory usage: 220.0+ bytes


In [57]:
df.head()

,Airline,FlightNumber,From_To,RecentDelays
0,KLM(!),10045,LoNDon_paris,"[23, 47]"
1,<Air France> (12),10055,MAdrid_miLAN,[]
2,(British Airways. ),10065,londON_StockhOlm,"[24, 43, 87]"
3,12. Air France,10075,Budapest_PaRis,[13]
4,"""Swiss Air""",10085,Brussels_londOn,"[67, 32]"


The From_To column would be better as two separate columns! Split each string on
the underscore delimiter _ to give a new temporary DataFrame with the correct values.
Assign the correct column names to this temporary DataFrame.

In [58]:
df_temp = df.From_To.str.split('_', expand=True)
df_temp.columns = ['From', 'To']

Notice how the capitalisation of the city names is all mixed up in this temporary
DataFrame. Standardise the strings so that only the first letter is uppercase (e.g.
"londON" should become "London".)

In [59]:
df_temp['From'] = df_temp['From'].str.capitalize()
df_temp['To'] = df_temp['To'].str.capitalize()
df_temp

,From,To
0,London,Paris
1,Madrid,Milan
2,London,Stockholm
3,Budapest,Paris
4,Brussels,London


Delete the From_To column from df and attach the temporary DataFrame from the
previous questions.

In [60]:
df.drop('From_To', axis=1,inplace=True)# Drop From_To column
df = df.join(df_temp) # Attach df_temp to df
df

,Airline,FlightNumber,RecentDelays,From,To
0,KLM(!),10045,"[23, 47]",London,Paris
1,<Air France> (12),10055,[],Madrid,Milan
2,(British Airways. ),10065,"[24, 43, 87]",London,Stockholm
3,12. Air France,10075,[13],Budapest,Paris
4,"""Swiss Air""",10085,"[67, 32]",Brussels,London


In the RecentDelays column, the values have been entered into the DataFrame as a
list. We would like each first value in its own column, each second value in its own
column, and so on. If there isn't an Nth value, the value should be NaN.
Expand the Series of lists into a DataFrame named delays, rename the columns delay_1,
delay_2, etc. and replace the unwanted RecentDelays column in df with delays.

In [61]:
delays = df['RecentDelays'].apply(pd.Series)
delays.columns = ['delay_{}'.format(n) for n in range(1, len(delays.columns)+1)]
df = df.drop('RecentDelays', axis=1).join(delays)
df

,Airline,FlightNumber,From,To,delay_1,delay_2,delay_3
0,KLM(!),10045,London,Paris,23.0,47.0,NaN
1,<Air France> (12),10055,Madrid,Milan,NaN,NaN,NaN
2,(British Airways. ),10065,London,Stockholm,24.0,43.0,87.0
3,12. Air France,10075,Budapest,Paris,13.0,NaN,NaN
4,"""Swiss Air""",10085,Brussels,London,67.0,32.0,NaN
